# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   

En este notebook se desarrolla el modelo teórico propuesto en el trabajo para la evaluación de armónicos esféricos mediante la utilización de computación cuántica.

Primeramente, es necesario instalar la librería "pennylane", que será utilizada para los circuitos cuánticos.

In [9]:
%pip install pennylane > o

Note: you may need to restart the kernel to use updated packages.


Importación de librerías utilizadas en este archivo.

In [10]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm


Ángulos utilizados para el cálculo de los armónicos esféricos. La primera coordenada corresponde con la coordenada azimutal y la segunda con la polar. Estos ángulos se corresponden con la posición de 64 electrodos de un EEG utilizando el "sistema 10-10" para elegir la posición de los mismos, considerando la cabeza como una esfera.

In [90]:
angles = [['Fc5.',-71,-21],
 ['Fc3.',-50,-28],
 ['Fc1.',-32,-45],
 ['Fcz.',23,90],
 ['Fc2.',32,45],
 ['Fc4.',50,28],
 ['Fc6.',71,21],
 ['C5..',-69,0],
 ['C3..',-46,0],
 ['C1..',23,0],
 ['Cz..',0,0],
 ['C2..',23,0],
 ['C4..',46,0],
 ['C6..',69,0],
 ['Cp5.',-71,21],
 ['Cp3.',-50,28],
 ['Cp1.',-32,45],
 ['Cpz.',23,-90],
 ['Cp2.',32,-45],
 ['Cp4.',50,-28],
 ['Cp6.',71,-21],
 ['Fp1.',-92,-72],
 ['Fpz.',92,90],
 ['Fp2.',92,72],
 ['Af7.',-92,-52],
 ['Af3.',-74,-67],
 ['Afz.',69,90],
 ['Af4.',74,67],
 ['Af8.',92,52],
 ['F7..',-92,-36],
 ['F5..',-75,-41],
 ['F3..',-60,-51],
 ['F1..',-50,-68],
 ['Fz..',46,90],
 ['F2..',50,68],
 ['F4..',60,51],
 ['F6..',75,41],
 ['F8..',92,36],
 ['Ft7.',-92,-18],
 ['Ft8.',92,18],
 ['T7..',-92,0],
 ['T8..',92,0],
 ['T9..',-115,0],
 ['T10.',115,0],
 ['Tp7.',-92,18],
 ['Tp8.',92,-18],
 ['P7..',-92,36],
 ['P5..',-75,41],
 ['P3..',-60,51],
 ['P1..',-50,68],
 ['Pz..',46,-90],
 ['P2..',50,-68],
 ['P4..',60,-51],
 ['P6..',75,-41],
 ['P8..',92,-36],
 ['Po7.',-92,54],
 ['Po3.',-74,67],
 ['Poz.',69,-90],
 ['Po4.',74,-67],
 ['Po8.',92,-54],
 ['O1..',-92,72],
 ['Oz..',92,-90],
 ['O2..',92,-72],
 ['Iz..',115,-90]
 ]

#Versión de los ángulos en radianes.
anglesRad = []
piFactor = math.pi/180
for x in angles:
    anglesRad.append([x[0], x[1] * piFactor, x[2] * piFactor])

In [34]:
dev = qml.device ("default.qubit", wires = nCubits)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])


def EvalAsocLegendre4(theta):
  """
  Evaluar los Polinomios asociados de legendre en un ángulo dado utilizando un circutio de 4 cúbits. Esto generará todos los polinomios asociados de Legendre hasta l = 4.

  Parámetros: 
    theta: ángulo en radiantes sobre el cual calcular los polinomios asociados de Legendre.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "P_l_m" indicando el polinomio asociado de Legendre al que hace referencia y cuyo valor es el resultado de la evaluación de dicho polinómio sobre el ángulo dado.
  """
  nCubits = 4
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] = (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] = -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] = 105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])

  #Polinomios de "m" negativo
  for l in range(1, nCubits + 1):
    for m in range(1, l+1):
      PolinomiosLegendre['P_' + str(l) + '_-' + str(m)] = ((-1)**(m) * (math.factorial(l-m)/math.factorial(l+m))) * PolinomiosLegendre['P_' + str(l) + '_' + str(m)]
  return PolinomiosLegendre

def EvalEsfericos4(theta, phi):
  """
  Realiza la evaluación de armónicos esféricos mediante cirucitos cuánticos de 4 cubits. Se obtiene el resultado de evalluar los armónicos esféricos para los ángulos dados hasta l = 4.

  Parámetros:
    theta: ángulo én radianes de la coordenada polar.
    phi: ángulo en radianes de la coordenada azimutal.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "Y_l_m" indicando el armónico esférico al que hace referencia y cuyo valor es el resultado de la evaluación de dicho armónico esféricos sobre los ángulos dados.
  """
  l = 4
  legendre = EvalAsocLegendre4(theta)
  AE = {}
  for k in range(l+1):
    for m in range(-k, k+1):
      coef =  math.sqrt(((2*k + 1)*math.factorial(k - (m))) / (4*math.pi * math.factorial(k+(m)))) #Coeficiente de normalización
      e = ExpPhi(phi, (m))
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(m)])
      AE['Y_' + str(k) + '_' + str(m)] = ( legendre_normalizado * e[0], legendre_normalizado * e[1])
  
  return AE

Se calculan los armónicos esféricos utilizando circuitos cuánticos según la propuesta del trabajo.

In [88]:
resultadoAE = {}
for x in anglesRad:
    AE = EvalEsfericos4(x[2], x[1])
    calculadosAE = []
    for armonicos in AE.values():
        calculadosAE.append(complex(armonicos[0], armonicos[1]))
    resultadoAE[x[0]] = calculadosAE
resultadoAE

{'Fc5.': [(0.28209479177387814+0j),
  (-0.04030990521889147-0.11706846521478603j),
  (0.45614974144993226+0j),
  (0.04030990521889147-0.11706846521478605j),
  (-0.03909184533201212+0.030541896871524803j),
  (-0.08414891426599758-0.2443861921559204j),
  (0.5092681972011995+0j),
  (0.08414891426599758-0.24438619215592042j),
  (-0.03909184533201211-0.030541896871524782j),
  (0.01610452747246998+0.010458402425479907j),
  (-0.09655770381554231+0.07543914611957607j),
  (-0.12661310386519542-0.3677111535015994j),
  (0.4730657269580536+0j),
  (0.12661310386519542-0.36771115350159944j),
  (-0.09655770381554232-0.07543914611957604j),
  (-0.016104527472469977+0.010458402425479905j),
  (0.0017657824014221794-0.007082166388397223j),
  (0.04510461487885328+0.029291279390576706j),
  (-0.17269219656629484+0.1349219309871189j),
  (-0.15979635980754117-0.4640823263660607j),
  (0.36392069764594415+0j),
  (0.15979635980754117-0.4640823263660608j),
  (-0.1726921965662948-0.13492193098711883j),
  (-0.045104

Se calculan los armónicos esféricos mediante la función correspondiente de la librería "scipy" de python. Hay que tener en cuenta que para esta función "theta" es la coordenada azimutal y "phi" la polar, al revés de lo considerado anteriormente.

In [91]:
nCubits = 4
m = []
v = []
for l in range(nCubits + 1):
    for n in range(-l, l+1):
        m.append(n)
        v.append(l)
resultadoSph = {}
for x in anglesRad:
    resultadoSph[x[0]]= sph_harm(m, v, x[1], x[2])
resultadoSph

{'Fc5.': array([ 0.28209479+0.j        ,  0.04030991+0.11706847j,
         0.45614974+0.j        , -0.04030991+0.11706847j,
        -0.03909185+0.0305419j ,  0.08414891+0.24438619j,
         0.5092682 +0.j        , -0.08414891+0.24438619j,
        -0.03909185-0.0305419j , -0.01610453-0.0104584j ,
        -0.0965577 +0.07543915j,  0.1266131 +0.36771115j,
         0.47306573+0.j        , -0.1266131 +0.36771115j,
        -0.0965577 -0.07543915j,  0.01610453-0.0104584j ,
         0.00176578-0.00708217j, -0.04510461-0.02929128j,
        -0.1726922 +0.13492193j,  0.15979636+0.46408233j,
         0.3639207 +0.j        , -0.15979636+0.46408233j,
        -0.1726922 -0.13492193j,  0.04510461-0.02929128j,
         0.00176578+0.00708217j]),
 'Fc3.': array([ 0.28209479+0.j        ,  0.10425994+0.12425216j,
         0.43141041+0.j        , -0.10425994+0.12425216j,
        -0.01478375+0.08384279j,  0.20584362+0.24531488j,
         0.42224287+0.j        , -0.20584362+0.24531488j,
        -0.01478375-0

Se comparan los resultados para obtener el error. Para simplificar se obtendrá el máximo error encontrado.

In [87]:
error = {}
for x in resultadoAE:
    error[x] = max(resultadoAE[x] - resultadoSph[x])
error

{'Fc5.': (0.3195927196150823-0.9281646527321212j),
 'Fc3.': (0.6194684046259251-0.7382536967115497j),
 'Fc1.': (0.6551582032402171-0.4093882818567938j),
 'Fcz.': (5.551115123125783e-17-2.7755575615628914e-17j),
 'Fc2.': (2.220446049250313e-16-2.7755575615628914e-17j),
 'Fc4.': (1.6653345369377348e-16-9.020562075079397e-17j),
 'Fc6.': (3.3306690738754696e-16+0j),
 'C5..': (6.900014960066974e-18-6.212801375759904e-18j),
 'C3..': (6.231121731408569e-18+6.452515441792753e-18j),
 'C1..': (8.256976436380707e-18-3.5048785552797355e-18j),
 'Cz..': (8.970052037653183e-18+0j),
 'C2..': (8.256976436380707e-18-3.5048785552797355e-18j),
 'C4..': (6.231121731408569e-18-6.452515441792749e-18j),
 'C6..': (6.900014960066974e-18+6.212801375759905e-18j),
 'Cp5.': (3.3306690738754696e-16+0j),
 'Cp3.': (1.6653345369377348e-16+6.938893903907228e-17j),
 'Cp1.': (2.220446049250313e-16+5.551115123125783e-17j),
 'Cpz.': (0.6360580828555744+0.26999063781875354j),
 'Cp2.': (0.6551582032402171+0.4093882818567939j)

Como se puede observar el máximo error es del orden de $10^{-16}$. Dicho error resulta despreciable y viene dado por la limitación en la precisión de los cálculos realizados mediante la librería "scipy".